In [1]:
import numpy as np
import pandas as pd

In [2]:
# READ GALAXY ID'S, MAGNITUDES AND MAGNITUDE ERRORS FROM FILE

# mags and errors
f = 'scratch.txt'
data = np.genfromtxt(f)

# id's
ngvsId = np.genfromtxt(f, dtype = 'S27', usecols = 0)

In [3]:
# READ FOREGROUND DUST REDDENINGS FROM ORIGINAL CATALOG FILE

df_ebv = pd.read_csv('finalCat_fixColNames.csv', index_col = 0, usecols = [0, 31])

In [4]:
# GENERATE HARMONIZED TABLES OF MAGNITUDES AND MAGNITUDE ERRORS (PREFER COG VALUES OVER GALFIT VALUES)

mags = np.zeros([data.shape[0], 5]) - 100.
magErrs = np.zeros([data.shape[0], 5]) - 100.

for i in range(data.shape[0]):
    if data[i, 1] > 0.:
        mags[i, :] = data[i, 1:10:2]
        magErrs[i, :] = data[i, 2:11:2]
    elif data[i, 11] > 0.:
        mags[i, :] = data[i, 11::2]
        magErrs[i, :] = data[i, 12::2]
        

In [5]:
# CONVERT MAGNITUDES AND THEIR ERRORS INTO FLUXES (AND ERRORS), AND OUTPUT TO FILE

f = open('observations.txt', 'w')

bands = ['u', 'g', 'r', 'i', 'z']

line = '# id' + (26 * ' ') + 'z  '
for i in range(5):
    line += '   flux_' + bands[i] + '      fluxErr_' + bands[i]
    
f.write(line + '\n')

c = [4.594, 3.560, 2.464, 1.813, 1.221]

fluxes = -100. * np.ones([data.shape[0], 5])
fluxErrs = -100. * np.ones([data.shape[0], 5])

for i in range(data.shape[0]):
    ID = str(ngvsId[i], 'utf-8')
    ebv = df_ebv.loc[str(ngvsId[i], 'utf-8'), 'e(B-V)']
    
    line = ID + '   0.0'
    for j in range(5):
        if mags[i, j] > 0.:
            fluxes[i, j] = 10.0 ** (23. - (mags[i, j] - (c[j] * ebv) - 31.09 + 48.6) / 2.5)
            fluxErrs[i, j] = magErrs[i, j] * fluxes[i, j] / 1.0857
            
        line += '   %9.3e   %9.3e' % (fluxes[i, j], fluxErrs[i, j])
        
    f.write(line + '\n')
    
f.close()